# 国土情報サービスなどの可視化
* 参考にしたページ
    * http://qiita.com/shima_x/items/fe29274d67de3a461524
    * http://sinhrks.hatenablog.com/entry/2015/06/14/215514
* 動作環境
    * anaconda3-4.0.0 を利用。他の anaconda のバージョンでも動作するはず。
    * gdal
      
      OSX の場合、OpenMP が標準の `gcc` に付属していないので、brew で入れる。
      ```
      brew install gdal  # version 1.11 when i installed it
      pip install gdal==1.11  # same as above
      ```
    *  その他のライブラリ
      ```
      pip install pyshp nkf
      pip install follium
      pip install seaborn  # Graph Visualization
      ```

## logger

In [ ]:
from logging import getLogger, StreamHandler, INFO, NullHandler
logger = getLogger(__name__)
# handler = StreamHandler()
handler = NullHandler()
handler.setLevel(INFO)
logger.setLevel(INFO)
logger.addHandler(handler)

logger.debug('hello')

## shapefile to geojson
http://nlftp.mlit.go.jp/ksj/ から区画用の shapefile を取得しておく。
ここでは大分県の区画を利用している。

In [ ]:
import os
import osgeo.ogr
import shapefile
import nkf
import json
from json import dumps

input_shape_file = "./N03-20150101_44_GML/N03-15_44_150101.shp"
intermediate_geojson_file = "./pyshp-oita-f02.json"

json 用に dict 型にする

In [ ]:
shape_reader = shapefile.Reader(input_shape_file)

# extract field names
fields = shape_reader.fields[1:]
field_names = [field[0] for field in fields]

out_buffer = []
for sr in shape_reader.shapeRecords():
    atr = dict(zip(field_names, sr.record))
    geom = sr.shape.__geo_interface__
    out_buffer.append(dict(type="Feature",
                       geometry=geom, properties=atr))

フィールド名に日本語があると、shift_jis のバイナリ文字列として認識してまうので utf-8 に変換する。

In [ ]:
for out_buffer_item in out_buffer:
    for prop_key, prop_val in out_buffer_item["properties"].items():
        logger.debug("key: {}, value: {}".format(prop_key, prop_val))        
        if type(prop_val) == bytes:
            logger.debug("str: {}".format(prop_val.decode('shift-jis')))
            out_buffer_item["properties"][prop_key] = prop_val.decode('shift-jis')
        if prop_key == "N03_007":
            logger.debug("Value of Key N03_007 change to {}".format("M{}".format(prop_val)))
            out_buffer_item["properties"][prop_key]= "M{}".format(prop_val)

json ファイルとして吐き出す

In [ ]:
geojson_str = dumps({"type": "FeatureCollection",
                     "features": out_buffer},
                    sort_keys=True, ensure_ascii=False, indent=2)

with open(intermediate_geojson_file, "w") as fh:
    logger.debug("output type: {}".format(type(geojson_str)))
    fh.write(geojson_str)

## Visualization via folium

In [ ]:
from IPython.display import HTML
import folium
import gdal
import urllib
import time
import pandas as pd
import numpy as np
import scipy as sp
from numba import jit
import seaborn
import matplotlib.pyplot as plt

%matplotlib inline

ipython notebook に HTML を吐き出す関数は follium 0.2 から不要になりました。

In [ ]:
# # folium.initialize_notebook()  # is not available

# def inline_map(m, out_html_str='tmp.html'):
#     # thanks to http://sinhrks.hatenablog.com/entry/2015/06/14/215514
#     m.create_map(path='tmp.html')
#     iframe = '<iframe srcdoc=\"{srcdoc}\" style=\"width: 100%; height: 400px; border: none\"></iframe>'
#     return HTML(iframe.format(srcdoc=m.HTML.replace('\"', '&quot;')))

とりあえず folium で HTML を履いてみる

In [ ]:
m = folium.Map(location=[33.2382026, 131.612535], zoom_start=8)
m.simple_marker([33.2382026, 131.612535], popup='Oita')

# This code is available at follium 0.1.x
# inline_map(m, "trial_oita.html")
m


### 地図を用いた可視化用にデータをとってくる
http://www.land.mlit.go.jp/webland/download 当たりから大分県の不動産取引価格情報を取得

In [ ]:
oita_webland_data_dir = "./webland_oita/"

### pandas で読み込む

フォルダ配下すべての csv を読み込んで結合する。
読み込むデータは不動産価格の取引情報（オープンデータ）

In [ ]:
arr_df_webland_oita = []
# find csv files and append to array
for file in os.listdir(oita_webland_data_dir):
    if file.endswith(".csv"):
        df_tmp = pd.read_csv(oita_webland_data_dir+file,
                             encoding="cp932",
                             index_col=0)
        arr_df_webland_oita.append(df_tmp)

#  all dataframes
df_webland_oita = pd.concat(arr_df_webland_oita, ignore_index=True)

In [ ]:
df_webland_oita.info()

In [ ]:
df_webland_oita.head()

坪単価の列に何故か null 値があるので自分で計算する。

In [ ]:
pd.value_counts(df_webland_oita["面積（㎡）"])

In [ ]:
def calc_price_per_tsubo(price_arr_values, area_arr_values):
    assert len(price_arr_values) == len(area_arr_values)
    return_arr = np.zeros([len(price_arr_values), ], dtype=np.float64)
    for i in range(len(price_arr_values)):
        try:
            float_area = float(area_arr_values[i])
            float_price = float(price_arr_values[i])
            return_arr[i] = float_price / (float_area * 0.3025)
        except Exception as err:
#             logger.debug(err)
            return_arr[i] = np.nan
    return return_arr

In [ ]:
df_webland_oita["price_per_tsubo"] = calc_price_per_tsubo(df_webland_oita["取引価格（総額）"].values,
                                                               df_webland_oita["面積（㎡）"].values)
df_webland_oita.describe()

In [ ]:
seaborn.distplot(df_webland_oita["price_per_tsubo"].dropna().values, kde=False, rug=True);

飛び抜けて高い値がいくつかあるので median で市町村コードで集計をかける

In [ ]:
df_oita_median_by_city = df_webland_oita.groupby("市区町村コード").median()


folium 0.2.1 でのバグのため、キーとなる列を文字列にする。

In [ ]:
df_oita_median_by_city["city_code"] = np.array(["M"+str(x) for x in df_oita_median_by_city.index.values])
df_oita_median_by_city.index = np.array(["M"+str(x) for x in df_oita_median_by_city.index.values])

In [ ]:
df_oita_median_by_city

In [ ]:
seaborn.regplot(x="坪単価", y="price_per_tsubo", data=df_oita_median_by_city)

In [ ]:
seaborn.distplot(df_oita_median_by_city["price_per_tsubo"].dropna().values, kde=False, rug=True);

### using dataframe and geojson visualization

In [ ]:
# map作成
m = folium.Map(location=[33.2382026, 131.612535], zoom_start=8)

# m.geo_json(geo_path=intermediate_geojson_file, data=df_oita_median_by_city,  # folium 0.1.x
m.choropleth(geo_path=intermediate_geojson_file, data=df_oita_median_by_city,
    columns=['city_code', 'price_per_tsubo'],
    key_on='feature.properties.N03_007',
    threshold_scale=[5000, 10000, 25000, 40000, 150000],
    fill_color='BuPu', reset=True,
    legend_name='price_per_tsubo')

# code using my `inline_map()` is available at follium 0.1.x
# inline_map(m)

m